# Overview
The famous ARIMA model is basically a Linear Regression model on a time series' own lags. So what if we replace the simple Linear Regression with a state-of-the-art tabular learning algorithm? This (known under the term *reduction*) is one example of useful features in [Sktime]. In this topic, we are going to use this library to implement a lot of forecasting tasks such as reduction, forecasting, backtesting and evaluation, in an unified way.

[Sktime]: https://www.sktime.org/en/stable/index.html

# 1. Forecasting interface

## 1.1. Forecasting reduction
In order to reduce a forecasting problem to a regression problem, we need to re-arrange the time series into a table so that itself ($\mathbf{y}_t$) becomes the label and its lags ($\mathbf{y}_{t-1},\mathbf{y}_{t-2},\dots$) become the features. There are three approaches to make prediction to multiple future time steps: recursive reduction, direct reduction and a hybrid between them, implemented in a single function,
<code style='font-size:13px'><a href='https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.compose.make_reduction.html'>make_reduction()</a></code>.

In [ ]:
import numpy as np
import pandas as pd
import janitor
import sktime
from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings; warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(['seaborn', 'seaborn-whitegrid'])
%config InlineBackend.figure_format = 'retina'

In [ ]:
dfTemp = pd.read_csv('data/air_quality.csv')
dfTemp = (
    dfTemp
    .to_datetime('time')
    .select_columns(['time', 'temp'])
    .also(lambda df: display(df.tail()))
    .set_index('time')
    .asfreq('h')
    # .to_period('h')
)

### Recursive strategy
This is the strategy used in traditional models such as ARIMA and ETS, it trains a model $f:\mathbf{y}_{t-1},\mathbf{y}_{t-2},\ldots\mapsto\mathbf{y}_t$. Denote the last observed value $y_T$, the model first predicts $\hat{y}_{T+1|T}$, then appends it to the original series to predict $\hat{y}_{T+2|T+1}$. This behaviour is repeated until all future time steps have been forecasted. This strategy has a significant weakness, is that it accumulates error each time it makes prediction to a new time step. Hence, requesting forecasts for a large horizon using this strategy is not a good idea, as it can saturate/converge and is no longer accurate.

<img src='image/forecast_reduction_recursive.png' style='height:300px; margin:20px auto 20px;'>

In [ ]:
nLags = 5
model = make_reduction(LGBMRegressor(), window_length=nLags, strategy='recursive')
model.fit(dfTemp)
model

In [ ]:
model.predict(fh=[1,2,3])

In [ ]:
pd.DataFrame({
    'lag': [f'lag{i}' for i in range(5, 0, -1)],
    'importance': model.estimator_.feature_importances_,
}).sort_values('importance')

In [ ]:
nTop = 10

df = pd.DataFrame({
    'feature': [f'lag{i}' for i in range(nLags, 0, -1)],
    'importance': model.estimator_.feature_importances_
})
df = df.sort_values('importance')
df = df.query("importance > 0")
df = df.tail(nTop)

fig, ax = plt.subplots(figsize=(5,1.5))
sns.barplot(data=df, x='importance', y='feature', color='teal')
plt.show()

### Direct strategy
This strategy, in the other hand, trains a separate model for each time step: $f_h:\mathbf{y}_{t-h},\mathbf{y}_{t-(h+1)},\ldots\mapsto\mathbf{y}_t$. As the number of models need to be train equals to the length of the forecasting horizon, this method is more computationally expensive but more accurate than recursive reduction. Keep in mind that making forecast to a far time step prevents using recent lags ($y_{t-1},\ldots,y_{t-(h-1)}$). In other the words, if $h$ is large then $f_h$ is trained on less features and thus has less prediction power. Also note that because this method is a combination of multiple models, we must specify the forecasting horizon when fitting it to the data.

<img src='image/forecast_reduction_direct.png' style='height:300px; margin:20px auto 20px;'>

In [ ]:
nLags = 10
model = make_reduction(LGBMRegressor(), window_length=nLags, strategy='direct', scitype='tabular-regressor')
model.fit(dfTemp, fh=[1,2,3])
model

In [ ]:
model.predict()

In [ ]:
model.estimators_[0].feature_importances_

### Hybrid strategy
Sktime also develops a hybrid method that combines two above strategies, call *dir-rec*. It also has a recursive behaviour, but after appending $\hat{y}_{T+h|T+(h-1)}$, it trains a new model to predict the next step $\hat{y}_{T+(h+1)|T+h}$. However, because both recursion reduction and direct reduction are not good at forecasting far horizons, *dir-rec* also suffers from this problem.

In [ ]:
nLags = 10
model = make_reduction(LGBMRegressor(), window_length=nLags, strategy='dirrec', scitype='tabular-regressor')
model.fit(dfTemp, fh=[1,2,3])
model

### Multivariate reducer
As Sktime currently does not support forecasting reduction for multivariate time series (where multiple time series grow together), we are going to use Skforecast's [multivariate forecasting] instead. Its class
<code style='font-size:13px'><a href='https://joaquinamatrodrigo.github.io/skforecast/0.5.1/api/ForecasterMultiSeries.html'>ForecasterAutoregMultiSeries</a></code>
assumes multiple time series are independent but follow the same patterns (for example, the sales of different products in the same store).

Another reducer being capable of modeling relationship between different time series are being developed. Its assumption is that, each time series depends not only on its own past values, but also on past values of other time series.

[multivariate forecasting]: https://joaquinamatrodrigo.github.io/skforecast/0.5.1/user_guides/multi-time-series-forecasting.html

In [ ]:
import numpy as np
import pandas as pd
import janitor

import matplotlib.pyplot as plt
plt.style.use(['seaborn', 'seaborn-whitegrid'])
%config InlineBackend.figure_format = 'retina'

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries

import warnings; warnings.filterwarnings('ignore')

In [ ]:
dfTemp = pd.read_csv('data/air_quality.csv')
dfTemp = (
    dfTemp
    .to_datetime('time')
    .select_columns(['time', 'temp', 'rel_humidity', 'abs_humidity'])
    .set_index('time')
    .asfreq('h')
)
dfTemp.tail()

In [ ]:
model = ForecasterAutoregMultiSeries(
    LGBMRegressor(max_depth=3), lags=5,
)
model.fit(dfTemp)
model.predict_interval(steps=7, level='temp')

In [ ]:
model.get_feature_importance()

## 1.2. Basic workflow

In [24]:
import datetime as dt
from sspipe import p, px
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import janitor
from sktime.forecasting.base import ForecastingHorizon as FH
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.ets import AutoETS

from sktime.utils.plotting import plot_series
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(['seaborn', 'seaborn-whitegrid'])
%config InlineBackend.figure_format = 'retina'

### Input format
The appropriate format of input data for Sktime is a Pandas Series or DataFrame with
<code style='font-size:13px'><a href=https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html>DatetimeIndex</a></code> and
<code style='font-size:13px'><a href=https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.PeriodIndex.html>PeriodIndex</a></code>.
There are different date and time [frequencies] available, but in this section we will demonstrate with a practical use case: business day frequency.
For non-time-series sequential data, Sktime also accepts integer index.

[frequencies]: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

In [3]:
dfQty = (
    pd.read_csv('data/weekly_quantity.csv')
    .eval("date = date.astype('datetime64')", engine='python')
    .set_index('date')
    .rename_axis(index=None)
    .asfreq('B')
    .fillna(1e-3)
    .to_period('B')
)
dfQty.tail()

,quantity
2015-06-26,58.07
2015-06-29,59.83
2015-06-30,47.84
2015-07-01,52.20
2015-07-02,44.57


### Forecasters
A pain point occurs in forecasting tasks is that different Python packages such as Statsmodels, PmdArima and Prophet use different fit and predict syntaxes. Sktime serves the role of an unified framework that wraps popular forecasting models so that Data Scientists can try different models with ease. Sktime supports all popular models, some with an automated version:
<code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.arima.AutoARIMA.html>AutoARIMA</a></code>
<code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.ets.AutoETS.html>AutoETS</a></code>
<code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.tbats.TBATS.html>TBATS</a></code>
<code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.fbprophet.Prophet.html>Prophet</a></code>.
Alongside with reducers introduced earlier, there a lot of options for Data Scientists to try.

In [88]:
model = AutoARIMA(sp=5)
model.fit(dfQty)

AutoARIMA(sp=5)

In [89]:
model.cutoff

Period('2015-07-02', 'B')

<b style='color:navy'><i class="fa fa-info-circle"></i>&nbsp; Note</b><br>
- Once the model is fitted, the last observed value is call the *offset*, indexed 0. It can be thought as the *now* state of the time series.
- We can either request future forecasts (*out-of-sample forecasting*) with the indices start from 1, or request historical fitted values (*in-sample prediction*) with negative indices. However, some models will return NA for in-sample prediction.

In [129]:
fh = np.arange(-1, 3)
model.predict(fh)

,quantity
2015-07-01,NaN
2015-07-02,NaN
2015-07-03,45.301294
2015-07-06,53.462003


### Forecasting horizon
The most simple way to request forecasts is using integer indices as above, but sometimes, we need a more pwerful tool. Sktime has a
<code style='font-size:13px'><a href=1>ForecastingHorizon</a></code>
class allowing us to specifiy either relative or absolute horizon.

[1]: https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.base.ForecastingHorizon.html

In [130]:
fhRel = np.arange(-1, 3) | p(FH)
display(fhRel)
model.predict(fhRel)

ForecastingHorizon([-1, 0, 1, 2], dtype='int64', is_relative=True)

,quantity
2015-07-01,NaN
2015-07-02,NaN
2015-07-03,45.301294
2015-07-06,53.462003


In [131]:
fhAbs = pd.period_range('2015-07-01', '2015-07-06', freq='B') | p(FH, is_relative=False)
model.predict(fhAbs)

,quantity
2015-07-01,NaN
2015-07-02,NaN
2015-07-03,45.301294
2015-07-06,53.462003


In [136]:
cutoff = pd.Period('2015-07-02', freq='B')
fhRel.to_absolute(cutoff) | p(display)

ForecastingHorizon(['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-06'], dtype='period[B]', is_relative=False)

### Update and predict
We have already known forecasting for far future points is not very accurate, as forecasting errors keep stacking up. For example, if we have daily data, then the forecast for this day next month $\hat{y}_{t+30|t}$ is not likely to have high quality, no matter how strong the algorithm we are using is. In this case, the approach to solve the problem is more important. A simple but good solution is re-training the model more frequently, like weekly or even daily. This strategy makes the cutoff closer to the query point: $\hat{y}_{t+7|t}$ (weekly) and $\hat{y}_{t+1|t}$ (daily).

The downside of frequent re-training is computation cost. Sktime offers a solution with lower training time but does not hurt the quality very much, that is updating. When calling the <code style='font-size:13px'>update()</code> method, the fitted model ingests new data and updates the cutoff to the last observed point. Let's see the behaviour of this method in action.

In [8]:
model = AutoARIMA()
model.fit(dfQty[:-5])
model.cutoff

Period('2015-06-25', 'B')

In [9]:
# after updating new 5 observations, the cutoff moves
model.update(dfQty[-5:])
model.cutoff

Period('2015-07-02', 'B')

In [10]:
fh = FH([1,2,3])
model.predict(fh)

,quantity
2015-07-03,50.533391
2015-07-06,53.435979
2015-07-07,52.105592


### Probabilistic prediction
Some Sktime forecasters have the ability to perform probabilistic prediction. Two common options are confidence interval (the user specifies a coverage) and distribution quantiles (the user specifies quantiles).

In [23]:
model.predict_interval(fh, coverage=0.9)

Coverage           
                  0.9           
                lower      upper
2015-07-03  34.881345  66.185437
2015-07-06  37.330719  69.541240
2015-07-07  35.862179  68.349005

In [ ]:
model.predict_quantiles(fh, alpha=[.05, .95])

Quantiles           
                 0.05       0.95
2015-07-03  34.881345  66.185437
2015-07-06  37.330719  69.541240
2015-07-07  35.862179  68.349005

### Exogeneous variables
Traditional statistical models, especially SARIMAX, have considered independent variables that potentially have a strong relationship with our time series. In terms of forecasting, the time series we want to make forecasts is called the *endogeneous* variable $\mathbf{y}$ and other independent time series are called *exogeneous* variables $\mathbf{x}_1,\mathbf{x}_2,\ldots$. When exogeneous variables are included in the forecaster, then the model becomes: $f:\mathbf{y}_{t-1},\mathbf{y}_{t-2},\ldots\mathbf{x}_1,\mathbf{x}_2,\ldots\mapsto\mathbf{y}$.

However, in the inference phase, it requres values of the exogeneous variables at future time steps that we are requesting forecast. This makes only special variables with appointed time (such as holidays and special events) a viable option. There is a workaround solution to involve economical data (which is much more useful), which is using their lags. For example, to forecast a variable $\mathbf{y}$ for the next 7 days, we can use $\mathbf{x}_{t-7}$ for training and leave $7$ latest values of $\mathbf{x}$ for inference.

In [106]:
dfTemp = pd.read_csv('data/air_quality.csv')
dfTemp = (
    dfTemp
    .to_datetime('time')
    .select_columns(['time', 'temp', '*humidity'])
    .set_index('time')
    .asfreq('h')
    .rename_axis(index=None)
    .loc['2005-01-15': '2005-03-15']
)
dfTemp.tail()

,temp,rel_humidity,abs_humidity
2005-03-15 19:00:00,18.0,36.0,0.7345
2005-03-15 20:00:00,16.4,41.9,0.7759
2005-03-15 21:00:00,14.9,47.6,0.8003
2005-03-15 22:00:00,13.8,49.6,0.7793
2005-03-15 23:00:00,13.2,51.6,0.7765


In [90]:
nForecast = 24

x = dfTemp.select_columns('*humidity')
x.index = x.index.shift(nForecast) # 16/01 to 16/03
xTrain = x[:-nForecast] # 16/01 to 15/03
xQuery = x[-nForecast:] # 16/03

y = dfTemp.select_columns('temp')
yTrain = y[nForecast:] # 16/01 to 15/03

In [92]:
model = AutoARIMA()
model.fit(yTrain, xTrain)

AutoARIMA()

In [104]:
fh = np.arange(1, nForecast+1)
yPred = model.predict(fh, xQuery)
yPred.iloc[[0,1,2,-3,-2,-1]]

,temp
2005-03-16 00:00:00,12.424923
2005-03-16 01:00:00,12.301092
2005-03-16 02:00:00,11.337493
2005-03-16 21:00:00,6.075261
2005-03-16 22:00:00,5.596284
2005-03-16 23:00:00,5.127052


# 2. Forecasting validation

## 2.1. Scoring metrics
Sktime supports a lot of [performance metrics] for time series forecasting. To better understand all of them, let's first break down each metric into three parts: *error* function, *squared* flag and *aggregation* strategy. We illustrate the break down process using a familiar metric, mean squared error.
- The type of error refers to how error is calculated for each observation. This is the centre of the metric and will be discussed later. In the case of MSE, the error is simply the difference between the true and forecasting values.
- Errors calculated in any way can be positive or negative, so they can eliminate each other. We can either take the *absolute* value or *squared* value to make the errors always positive. The choice of square function penalizes large errors.
- Two above functions give us the error for each observation, so we need a function to aggregate the result for the entire target variable. Most popular options are (arithmetic) *mean* and *median*, but people also use *geometric mean*.

[performance metrics]: https://www.sktime.org/en/stable/api_reference/performance_metrics.html

### Baseline forecasters
Many evaluation metrics for time series forecasting involve a [baseline model] (also known as benchmark model) in its calculation. The reason is very simple: because these baseline forecasters really make sense. Sktime implements all three strategies via the class
<code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.naive.NaiveForecaster.html>NaiveForecaster</a></code>.

- *Average model*: the forecast for all future values are equal to the average of observed values:

$$\hat{y}_t=\frac{y_1+y_2+\ldots+y_T}{T}$$

- *Drift model*: we draw a line from the first to the last observed point, then extrapolate it into the future:

$$\hat{y}_t=y_1+t\left(\frac{y_T-y_1}{T-1}\right)$$

- *Naive model*: we take the last observed value of the same season (time step $t-m$, with $m$ the seasonal period) as the forecast at a time step. If data has no seasonality then $m=1$. The formulation is:

$$\hat{y}_t=y_{t-m} $$

[baseline model]: https://en.wikipedia.org/wiki/Forecasting#Categories_of_forecasting_methods

### Difference error
$$e_t=y_t-\hat{y}_t$$

Two popular metrics of this type are [MSE] and [MAE], also have been seen in evaluating regression problems. These two metrics are scale-dependent, so that they can express in the same unit as the time series. Scale-dependence is also a disadvantage, as we cannot use these metrics to decide if a model is good enough, since its range is $(0,\infty)$.

An approach to solve this is to take the ratio between the metric of our Machine Learning model (like $\text{MSE}$) and the metric of a naive baseline forecaster ($\text{MSE}^*$). This metric is called *Relative Loss*, computed as:

$$\text{RelativeMSE}=\frac{\text{MSE}}{\text{MSE}^*} $$

[MSE]: https://en.wikipedia.org/wiki/Mean_squared_error
[MAE]: https://en.wikipedia.org/wiki/Mean_absolute_error

### Percentage error
$$e_t=\frac{y_t-\hat{y}_t}{y_t}$$

[MAPE] is the most popular metric for this type of error. It is *scale-independent* and *high-interpretable*, so it can be considered the standard metric for forecasting tasks. However, percentage error suffers from many known issues:
- It cannot be used when $y_t=0$ and tends towards infinity when $y_t\approx0$.
- It is meaningless for data in [interval scale] (for example, temperatures), in which the ratio is not defined.
- An controversial issue that percentage error is said to have, is its asymmetry. The symmetric counterpart is given by:

$$e_t=\frac{2(y_t-\hat{y}_t)}{|y_t|+|\hat{y}_t|} $$

In this formula, the nominator is multiplied by $2$ as a compensation for larger denominator. The symmetric variant of MAPE is called [sMAPE]. When we switch the position of $y_t$ and $\hat{y}_t$, sMAPE does not change like MAPE. We can see their behaviors in action:

|Actual|Forecast|Difference|MAPE|sMAPE|
|:-:|:-:|:-:|:-:|:-:|
|100|80|20|20%|22%|
|80|100|-20|25%|22%|

However, the above case is not quite a problem of MAPE because we would never compare values that way. In fact, sMAPE symmetrizes one case but makes another case asymmetric:

|Actual|Forecast|Difference|MAPE|sMAPE|
|:-:|:-:|:-:|:-:|:-:|
|100|80|20|20%|22%|
|100|120|-20|20%|18%|

The later case, in the other hand, shows a critical issue of sMAPE: it penalizes forecasts that are lower than ground truth. So that it's likely to encourages higher forecasts and can lead to biases. Overall, MAPE is still the better metric in my opinion.

[MAPE]: https://en.wikipedia.org/wiki/Mean_absolute_percentage_error
[sMAPE]: https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error
[interval scale]: https://en.wikipedia.org/wiki/Level_of_measurement#Interval_scale

### Relative error
$$e_t=\frac{y_t-\hat{y}_t}{y_t-\hat{y}_t^*} $$

As the name states, this metric expresses the relative error between our forecaster and a benchmark model. Due to the scale-free property, it is recommended to take the absolute value of relative error rather than squared value. In addition, geometric mean and median is prefered to arithmetic mean, making GMRAE and MdRAE two most popular metrics using this type of error.

### Scaled error
$$e_t=\frac{y_t-\hat{y}_t}{\text{MAE}^*}$$

This is another error type makes use of a benchmark model. Specifically, it computes $\text{MAE}^*$ on the naive forecaster (forecast using last observed value, or using value of the same season in the last period). The most common metric of this type, [MASE], a along with MAPE are considered the best metrics for forecasting problems. MASE has the following desired properties:
- It is scale-free, so can be used for different data with different scales
- It is symmetric, covers both cases that either MAPE or sMAPE suffers from
- It is stable when $y_t\approx0$
- It is easy to interpret

[MASE]: https://en.wikipedia.org/wiki/Mean_absolute_scaled_error

### Implementation
Sktime provides two APIs to compute metrics, function interface and class interface. There are some notes when implementing evaluation metrics in Sktime:
- For metrics with different variants and settings (such as sMAPE, RMSE and RelMSE), it is recommended to use the class interface. Otherwise, the function interface will be more convenient.
- Metrics using *relative error* (such as MdRAE and GMRAE) require the forecasted values of the baseline model to be provided via the parameter
<code style='font-size:13px'>y_pred_benchmark</code>.
This could make the forecasting solution more complicated.
- Metrics using *scaled error* (such as MASE) require the training values to be provided via the parameter
<code style='font-size:13px'>y_train</code>.
Same as the case of relative error, this behaviour can make the validation phase more complicated.

In [ ]:
from sktime.performance_metrics.forecasting import (
    mean_absolute_error as MAE,
    mean_squared_error as MSE,
    mean_absolute_percentage_error as MAPE,
    median_absolute_percentage_error as MdAPE,
    median_relative_absolute_error as MdRAE,
    geometric_mean_relative_absolute_error as GMRAE,
    mean_absolute_scaled_error as MASE,
    
    RelativeLoss,
    MeanSquaredError,
    MeanAbsolutePercentageError,
)

MSE = MeanSquaredError()
RMSE = MeanSquaredError(square_root=True)
MAPE = MeanAbsolutePercentageError()
sMAPE = MeanAbsolutePercentageError(symmetric=True)
RelMSE = RelativeLoss(relative_loss_function=MSE)

RelMSE.greater_is_better

## 2.2. Backtesting
[Backtesting](https://en.wikipedia.org/wiki/Backtesting) is the validation strategy dedicated for time series forecasting. The idea is to go back to the past, try different forecasters to decide which one best fits to our data. Backtesting can be either holdout validation or cross validation, but holdout validation is significantly biased for time series becasue data changes over time, so we highly recommend using cross validation.

In [ ]:
from sspipe import p, px
import numpy as np
import pandas as pd
import janitor
import sktime
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_selection import CutoffSplitter, SlidingWindowSplitter, ExpandingWindowSplitter
from sktime.forecasting.model_selection import ForecastingGridSearchCV, ForecastingRandomizedSearchCV
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.model_evaluation import evaluate

import warnings; warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(['seaborn', 'seaborn-whitegrid'])
%config InlineBackend.figure_format = 'retina'

In [ ]:
dfTemp = pd.read_csv('data/air_quality.csv')
dfTemp = (
    dfTemp
    .to_datetime('time')
    .select_columns(['time', 'temp'])
    .set_index('time')
    .asfreq('h')
)
dfTemp.tail()

### Cross validation
Currently, Sktime supports three splitting strategies for rolling backtesting:
- <code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.CutoffSplitter.html>CutoffSplitter</a></code>
splits the time series at provided cutoff points with a fixed training window length.
- <code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.SlidingWindowSplitter.html>SlidingWindowSplitter</a></code>
is very much like cutoff splitter, but it uses all possible cutoffs rather than requires users to explicitly specify the locations. Visually, we can think about this strategy as *sliding* the observed window incrementally. There can be a large number of splits returned by this strategy, we can easily reduce it by setting a high sliding velocity.
- <code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.ExpandingWindowSplitter.html>ExpandingWindowSplitter</a></code>
is like sliding window strategy, but the origin of the training window is fixed rather than its length. So, the training window is more likely of *expanding* rather than *sliding*, thus the name of the strategy.

In [ ]:
def printSplitIndex(splitter, limit=5):
    splitter = list(splitter)[:limit]
    for i, split in enumerate(splitter):
        trainStart, trainEnd = split[0][[0,-1]]
        validStart, validEnd = split[1][[0,-1]]
        print(
            f'Split #{i+1}' ': '
            f'train [{trainStart}, {trainEnd}]' ', '
            f'valid [{validStart}, {validEnd}]'
        )

In [ ]:
splitter = CutoffSplitter(fh=[1,2,3], window_length=10, cutoffs=np.array([100,200]))
splitter.split(dfTemp) | p(list) | p(printSplitIndex)

In [ ]:
splitter = SlidingWindowSplitter(fh=[1,2,3,4,5], window_length=10, initial_window=100, step_length=50)
splitter.split(dfTemp) | p(list) | p(printSplitIndex)

In [ ]:
splitter = ExpandingWindowSplitter(fh=[1,2,3,4,5], initial_window=100, step_length=50)
splitter.split(dfTemp) | p(list) | p(printSplitIndex)

### Hyperparameter tuning
Like in Scikit-learn, Sktime provides two convinient classes that combine hyperparameter tuning with cross validation, 
<code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.ForecastingRandomizedSearchCV.html>ForecastingRandomizedSearchCV</a></code> and
<code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.ForecastingGridSearchCV.html>ForecastingGridSearchCV</a></code>.
When implementing, there are two points to notice:
- Before performing validation splitting, we need to define a test set first. This is supported in Sktime using
<code style='font-size:13px'><a href=https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.temporal_train_test_split.html>temporal_train_test_split()</a></code>, a wrapper function of Sklearn's counterpart without shuffling.
- As time series data will be quickly outdated, it is necessary to perform either refitting (more accurate) or updating (faster) after each time moving to a new training window. Implemented in both validators via the paramter <code style='font-size:13px'>strategy</code>.

In [ ]:
yTrain, yTest = temporal_train_test_split(dfTemp, fh=[1,2,3])

is a special type of cross-validation which involves moving backward in time step-by-step. Function <code style='font-size:13px'><a href='https://joaquinamatrodrigo.github.io/skforecast/0.5.0/api/model_selection.html#skforecast.model_selection.model_selection.backtesting_forecaster'>backtesting_forecaster</a></code> wrapped 3 types of backtest through 2 params  `fixed_train_size` and `refit`:
- Backtesting with refit and increasing training size (fixed origin): The model is trained each time before making predictions, the test size is fixed and the training set increases sequentially. 
- Backtesting with refit and fixed training size (rolling origin): Both the training and testing set rolls forward, therefore, the size of training remains constant. 
- Backtesting without refit: The model is trained only once and not updating sequentially.Thus, this strategy has the advantage of being much faster. However, the model does not incorporate the latest information available, so it may lose predictive capacity over time.

## 2.3. Hyperparameter tuning

### Grid search  

[GridSearch](https://joaquinamatrodrigo.github.io/skforecast/0.5.0/api/model_selection.html#skforecast.model_selection.model_selection.grid_search_forecaster) and [RandomSearch](https://joaquinamatrodrigo.github.io/skforecast/0.5.0/api/model_selection.html#skforecast.model_selection.model_selection.random_search_forecaster) has similar syntax

### Bayesian search 

<code style='font-size:13px'><a href='https://joaquinamatrodrigo.github.io/skforecast/0.5.0/api/model_selection.html#skforecast.model_selection.model_selection.bayesian_search_forecaster'>bayesian_search_forecaster</a></code> supports *optuna* and *skpot* engine


# References
- *sktime.org - [Forecasting with Sktime](https://nbviewer.org/github/sktime/sktime/blob/4194aacaa105ef59807ea56c1bb3497b37eb1613/examples/01_forecasting.ipynb)*
- *typethepipe.com - [Symmetric MAPE is not symmetric](https://typethepipe.com/post/symmetric-mape-is-not-symmetric/)*
- https://nbviewer.org/github/sktime/sktime/blob/4194aacaa105ef59807ea56c1bb3497b37eb1613/examples/01_forecasting.ipynb
- https://sites.google.com/site/nttrungmtwiki/home/it/data-mining/time-series-analysis/time-series-forecasting-performance-metrics-comparison
- https://www.sktime.org/en/stable/api_reference/performance_metrics.html
- https://towardsdatascience.com/time-series-forecast-error-metrics-you-should-know-cc88b8c67f27
- https://www.relexsolutions.com/resources/measuring-forecast-accuracy/
- https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.mean_absolute_percentage_error.html
- https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.performance_metrics.forecasting.mean_absolute_scaled_error.html#
- https://arxiv.org/pdf/1909.07872.pdf
- https://arxiv.org/pdf/2005.08067.pdf

---
*&#9829; By Quang Hung x Thuy Linh &#9829;*